In [1]:
# 挂载google云硬盘
import os
from google.colab import drive
drive.mount('/content/gdrive')

# 更改工程的运行文件夹
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Task2")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### 1. Utils 辅助函数单元

In [0]:
import codecs
import json
from os.path import join
import pickle
import os
import re

################# Load and Save Data ################

def load_json(rfdir, rfname):
    with codecs.open(join(rfdir, rfname), 'r', encoding='utf-8') as rf:
        return json.load(rf)


def dump_json(obj, wfpath, wfname, indent=None):
    with codecs.open(join(wfpath, wfname), 'w', encoding='utf-8') as wf:
        json.dump(obj, wf, ensure_ascii=False, indent=indent)



def dump_data(obj, wfpath, wfname):
    with open(os.path.join(wfpath, wfname), 'wb') as wf:
        pickle.dump(obj, wf)


def load_data(rfpath, rfname):
    with open(os.path.join(rfpath, rfname), 'rb') as rf:
        return pickle.load(rf)

    
################# Random Walk ################

import random
class MetaPathGenerator:
    def __init__(self):
        self.paper_author = dict()
        self.author_paper = dict()
        self.paper_org = dict()
        self.org_paper = dict()
        self.paper_conf = dict()
        self.conf_paper = dict()

    def read_data(self, dirpath):
        temp=set()

        with open(dirpath + "/paper_org.txt", encoding='utf-8') as pafile:
            for line in pafile:
                temp.add(line)                       
        for line in temp: 
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    p, a = toks[0], toks[1]
                    if p not in self.paper_org:
                        self.paper_org[p] = []
                    self.paper_org[p].append(a)
                    if a not in self.org_paper:
                        self.org_paper[a] = []
                    self.org_paper[a].append(p)
        temp.clear()

              
        with open(dirpath + "/paper_author.txt", encoding='utf-8') as pafile:
            for line in pafile:
                temp.add(line)                       
        for line in temp: 
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    p, a = toks[0], toks[1]
                    if p not in self.paper_author:
                        self.paper_author[p] = []
                    self.paper_author[p].append(a)
                    if a not in self.author_paper:
                        self.author_paper[a] = []
                    self.author_paper[a].append(p)
        temp.clear()
        
                
        with open(dirpath + "/paper_conf.txt", encoding='utf-8') as pcfile:
            for line in pcfile:
                temp.add(line)                       
        for line in temp: 
                toks = line.strip().split("\t")
                if len(toks) == 2:
                    p, a = toks[0], toks[1]
                    if p not in self.paper_conf:
                        self.paper_conf[p] = []
                    self.paper_conf[p].append(a)
                    if a not in self.conf_paper:
                        self.conf_paper[a] = []
                    self.conf_paper[a].append(p)
        temp.clear()
                    
        # print ("#papers ", len(self.paper_conf))      
        # print ("#authors", len(self.author_paper))
        # print ("#org_words", len(self.org_paper))
        # print ("#confs  ", len(self.conf_paper)) 
    
    def generate_WMRW(self, outfilename, numwalks, walklength):
        outfile = open(outfilename, 'w')
        for paper0 in self.paper_conf: 
            for j in range(0, numwalks): #wnum walks
                paper=paper0
                outline = ""
                i=0
                while(i<walklength):
                    i=i+1    
                    if paper in self.paper_author:
                        authors = self.paper_author[paper]
                        numa = len(authors)
                        authorid = random.randrange(numa)
                        author = authors[authorid]
                        
                        papers = self.author_paper[author]
                        nump = len(papers)
                        if nump >1:
                            paperid = random.randrange(nump)
                            paper1 = papers[paperid]
                            while paper1 == paper:
                                paperid = random.randrange(nump)
                                paper1 = papers[paperid]
                            paper = paper1
                            outline += " " + paper           
                        
                    if paper in self.paper_org:
                        words = self.paper_org[paper]
                        numw = len(words)
                        wordid = random.randrange(numw) 
                        word = words[wordid]
                    
                        papers = self.org_paper[word]
                        nump = len(papers)
                        if nump >1:
                            paperid = random.randrange(nump)
                            paper1 = papers[paperid]
                            while paper1 == paper:
                                paperid = random.randrange(nump)
                                paper1 = papers[paperid]
                            paper = paper1
                            outline += " " + paper  
                            
                outfile.write(outline + "\n")
        outfile.close()
        
        # print ("walks done")
        
################# Compare Lists ################

def tanimoto(p,q):
    c = [v for v in p if v in q]
    return float(len(c) / (len(p) + len(q) - len(c)))



################# Paper similarity ################

def generate_pair(pubs,outlier): ##求匹配相似度
    dirpath = 'gene'
    
    paper_org = {}
    paper_conf = {}
    paper_author = {}
    paper_word = {}
    
    temp=set()
    with open(dirpath + "/paper_org.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_org:
                paper_org[p] = []
            paper_org[p].append(a)
    temp.clear()
    
    with open(dirpath + "/paper_conf.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_conf:
                paper_conf[p]=[]
            paper_conf[p]=a
    temp.clear()
    
    with open(dirpath + "/paper_author.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_author:
                paper_author[p] = []
            paper_author[p].append(a)
    temp.clear()
       
    with open(dirpath + "/paper_word.txt", encoding='utf-8') as pafile:
        for line in pafile:
            temp.add(line)                       
    for line in temp: 
        toks = line.strip().split("\t")
        if len(toks) == 2:
            p, a = toks[0], toks[1]
            if p not in paper_word:
                paper_word[p] = []
            paper_word[p].append(a)
    temp.clear()
    
    
    paper_paper = np.zeros((len(pubs),len(pubs)))
    for i,pid in enumerate(pubs):
        if i not in outlier:
            continue
        for j,pjd in enumerate(pubs):
            if j==i:
                continue
            ca=0
            cv=0
            co=0
            ct=0
          
            if pid in paper_author and pjd in paper_author:
                ca = len(set(paper_author[pid])&set(paper_author[pjd]))*1.5
            if pid in paper_conf and pjd in paper_conf and 'null' not in paper_conf[pid]:
                cv = tanimoto(set(paper_conf[pid]),set(paper_conf[pjd]))
            if pid in paper_org and pjd in paper_org:
                co = tanimoto(set(paper_org[pid]),set(paper_org[pjd]))
            if pid in paper_word and pjd in paper_word:
                ct = len(set(paper_word[pid])&set(paper_word[pjd]))/3
                    
            paper_paper[i][j] =ca+cv+co+ct
            
    return paper_paper

    
        
################# Evaluate ################
        
def pairwise_evaluate(correct_labels,pred_labels):
    TP = 0.0  # Pairs Correctly Predicted To SameAuthor
    TP_FP = 0.0  # Total Pairs Predicted To SameAuthor
    TP_FN = 0.0  # Total Pairs To SameAuthor

    for i in range(len(correct_labels)):
        for j in range(i + 1, len(correct_labels)):
            if correct_labels[i] == correct_labels[j]:
                TP_FN += 1
            if pred_labels[i] == pred_labels[j]:
                TP_FP += 1
            if (correct_labels[i] == correct_labels[j]) and (pred_labels[i] == pred_labels[j]):
                TP += 1

    if TP == 0:
        pairwise_precision = 0
        pairwise_recall = 0
        pairwise_f1 = 0
    else:
        pairwise_precision = TP / TP_FP
        pairwise_recall = TP / TP_FN
        pairwise_f1 = (2 * pairwise_precision * pairwise_recall) / (pairwise_precision + pairwise_recall)
    return pairwise_precision, pairwise_recall, pairwise_f1


################# Save Paper Features ################

def save_relation(name_pubs_raw, name): # 保存论文的各种feature
    name_pubs_raw = load_json('genename', name_pubs_raw)
    ## trained by all text in the datasets. Training code is in the cells of "train word2vec"
    save_model_name = "word2vec/Aword2vec.model"
    model_w = word2vec.Word2Vec.load(save_model_name)
    
    r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
    stopword = ['at','based','in','of','for','on','and','to','an','using','with','the','by','we','be','is','are','can']
    stopword1 = ['university','univ','china','department','dept','laboratory','lab','school','al','et',
                 'institute','inst','college','chinese','beijing','journal','science','international']
    
    f1 = open ('gene/paper_author.txt','w',encoding = 'utf-8')
    f2 = open ('gene/paper_conf.txt','w',encoding = 'utf-8')
    f3 = open ('gene/paper_word.txt','w',encoding = 'utf-8')
    f4 = open ('gene/paper_org.txt','w',encoding = 'utf-8')

    
    taken = name.split("_")
    name = taken[0] + taken[1]
    name_reverse = taken[1]  + taken[0]
    if len(taken)>2:
        name = taken[0] + taken[1] + taken[2]
        name_reverse = taken[2]  + taken[0] + taken[1]
    
    authorname_dict={}
    ptext_emb = {}  
    
    tcp=set()  
    for i,pid in enumerate(name_pubs_raw):
        
        pub = name_pubs_raw[pid]
        
        #save authors
        org=""
        for author in pub["authors"]:
            authorname = re.sub(r,'', author["name"]).lower()
            taken = authorname.split(" ")
            if len(taken)==2: ##检测目前作者名是否在作者词典中
                authorname = taken[0] + taken[1]
                authorname_reverse = taken[1]  + taken[0] 
            
                if authorname not in authorname_dict:
                    if authorname_reverse not in authorname_dict:
                        authorname_dict[authorname]=1
                    else:
                        authorname = authorname_reverse 
            else:
                authorname = authorname.replace(" ","")
            
            if authorname!=name and authorname!=name_reverse:
                f1.write(pid + '\t' + authorname + '\n')
        
            else:
                if "org" in author:
                    org = author["org"]
                    
                    
        #save org 待消歧作者的机构名
        pstr = org.strip()
        pstr = pstr.lower() #小写
        pstr = re.sub(r,' ', pstr) #去除符号
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip() #去除多余空格
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        pstr=set(pstr)
        for word in pstr:
            f4.write(pid + '\t' + word + '\n')

        
        #save venue
        pstr = pub["venue"].strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword1]
        pstr = [word for word in pstr if word not in stopword]
        for word in pstr:
            f2.write(pid + '\t' + word + '\n')
        if len(pstr)==0:
            f2.write(pid + '\t' + 'null' + '\n')

            
        #save text
        pstr = ""    
        keyword=""
        if "keywords" in pub:
            for word in pub["keywords"]:
                keyword=keyword+word+" "
        pstr = pstr + pub["title"]
        pstr=pstr.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>1]
        pstr = [word for word in pstr if word not in stopword]
        for word in pstr:
            f3.write(pid + '\t' + word + '\n')
        
        #save all words' embedding
        pstr = keyword + " " + pub["title"] + " " + pub["venue"] + " " + org
        if "year" in pub:
              pstr = pstr +  " " + str(pub["year"])
        pstr=pstr.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        pstr = pstr.split(' ')
        pstr = [word for word in pstr if len(word)>2]
        pstr = [word for word in pstr if word not in stopword]
        pstr = [word for word in pstr if word not in stopword1]

        words_vec=[]
        for word in pstr:
            if (word in model_w):
                words_vec.append(model_w[word])
        if len(words_vec)<1:
            words_vec.append(np.zeros(100))
            tcp.add(i)
            #print ('outlier:',pid,pstr)
        ptext_emb[pid] = np.mean(words_vec,0)
        
    #  ptext_emb: key is paper id, and the value is the paper's text embedding
    dump_data(ptext_emb,'gene','ptext_emb.pkl')
    # the paper index that lack text information
    dump_data(tcp,'gene','tcp.pkl')
            
 
    f1.close()
    f2.close()
    f3.close()
    f4.close()

### 1.1 数据预处理及强规则匹配函数

In [0]:
import re
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer

# 预处理名字
def precessname(name):   
    name = name.lower().replace(' ', '_')
    name = name.replace('.', '_')
    name = name.replace('-', '')
    name = re.sub(r"_{2,}", "_", name) 
    return name

# 预处理机构,简写替换，
def preprocessorg(org):
    if org != "":
        org = org.replace('Sch.', 'School')
        org = org.replace('Dept.', 'Department')
        org = org.replace('Coll.', 'College')
        org = org.replace('Inst.', 'Institute')
        org = org.replace('Univ.', 'University')
        org = org.replace('Lab ', 'Laboratory ')
        org = org.replace('Lab.', 'Laboratory')
        org = org.replace('Natl.', 'National')
        org = org.replace('Comp.', 'Computer')
        org = org.replace('Sci.', 'Science')
        org = org.replace('Tech.', 'Technology')
        org = org.replace('Technol.', 'Technology')
        org = org.replace('Elec.', 'Electronic')
        org = org.replace('Engr.', 'Engineering')
        org = org.replace('Aca.', 'Academy')
        org = org.replace('Syst.', 'Systems')
        org = org.replace('Eng.', 'Engineering')
        org = org.replace('Res.', 'Research')
        org = org.replace('Appl.', 'Applied')
        org = org.replace('Chem.', 'Chemistry')
        org = org.replace('Prep.', 'Petrochemical')
        org = org.replace('Phys.', 'Physics')
        org = org.replace('Phys.', 'Physics')
        org = org.replace('Mech.', 'Mechanics')
        org = org.replace('Mat.', 'Material')
        org = org.replace('Cent.', 'Center')
        org = org.replace('Ctr.', 'Center')
        org = org.replace('Behav.', 'Behavior')
        org = org.replace('Atom.', 'Atomic')
        org = org.split(';')[0]  # 多个机构只取第一个
    return org

#正则去标点
def etl(content):
    content = re.sub("[\s+\.\!\/,;$%^*(+\"\')]+|[+——()?【】“”！，。？、~@#￥%……&*（）]+", " ", content)
    content = re.sub(r" {2,}", " ", content)
    return content

def get_org(co_authors, author_name):
    for au in co_authors:
        name = precessname(au['name'])
        name = name.split('_')
        if ('_'.join(name) == author_name or '_'.join(name[::-1]) == author_name) and 'org' in au:
            return au['org']
    return ''


# 基于机构和作者的联合匹配函数
def disambiguate_by_cluster(papers, pubs_raw):
    coauther_orgs = []
    for p_id in papers:
        paper = pubs_raw[p_id]
        authors = paper['authors'] 
        names = [precessname(paper_author['name']) for paper_author in authors]
        orgs = [preprocessorg(paper_author['org']) for paper_author in authors if 'org' in paper_author]  
        abstract = paper["abstract"] if 'abstract' in paper else ''
        coauther_orgs.append(etl(' '.join(names + orgs) + ' '+ abstract))         
    tfidf = TfidfVectorizer().fit_transform(coauther_orgs)
    sim_mertric = pairwise_distances(tfidf, metric='cosine')

    return sim_mertric

### 2. 清洗文本，训练语义向量

In [0]:
import codecs
import json
from os.path import join
import pickle
import os
import re
from tqdm import tqdm_notebook

# 加载训练集和验证集的文本数据，抽出语义特征形成语料集
pubs_raw = load_json("train","train_pub.json")
pubs_raw1 = load_json("sna_data","sna_valid_pub.json")

r = '[!“”"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—～’]+'
f1 = open ('gene/all_text.txt','w',encoding = 'utf-8')

# 整理训练集数据
for i,pid in tqdm_notebook(enumerate(pubs_raw), 
                           total=len(pubs_raw), desc='TrainSet Corpus:'):
    pub = pubs_raw[pid]
    
    for author in pub["authors"]:
        if "org" in author:
                org = author["org"]
                pstr = org.strip()
                pstr = pstr.lower()
                pstr = re.sub(r,' ', pstr)
                pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
                f1.write(pstr+'\n')
            
    title = pub["title"]
    pstr=title.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
    if "abstract" in pub and type(pub["abstract"]) is str:
        abstract = pub["abstract"]
        pstr=abstract.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        f1.write(pstr+'\n')
        
    venue = pub["venue"]
    pstr=venue.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')

# 整理验证集数据    
for i,pid in tqdm_notebook(enumerate(pubs_raw1), 
                           desc='ValidSet Corpus:', total=len(pubs_raw1)):
    pub = pubs_raw1[pid]
    
    for author in pub["authors"]:
        if "org" in author:
                org = author["org"]
                pstr = org.strip()
                pstr = pstr.lower()
                pstr = re.sub(r,' ', pstr)
                pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
                f1.write(pstr+'\n')
            
    title = pub["title"]
    pstr=title.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
    
    if "abstract" in pub and type(pub["abstract"]) is str:
        abstract = pub["abstract"]
        pstr=abstract.strip()
        pstr = pstr.lower()
        pstr = re.sub(r,' ', pstr)
        pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
        f1.write(pstr+'\n')
        
    venue = pub["venue"]
    pstr=venue.strip()
    pstr = pstr.lower()
    pstr = re.sub(r,' ', pstr)
    pstr = re.sub(r'\s{2,}', ' ', pstr).strip()
    f1.write(pstr+'\n')
        
f1.close()

In [0]:
from gensim.models import word2vec

sentences = word2vec.Text8Corpus(r'gene/all_text.txt')
model = word2vec.Word2Vec(sentences, size=100,negative =5, min_count=2, window=5)
model.save('word2vec/Aword2vec.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### 3. 在训练集上进行参数调优+指标预估

In [0]:
import re
from gensim.models import word2vec
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from tqdm import tqdm_notebook
import warnings
 
# 忽略警告
warnings.filterwarnings('ignore')


pubs_raw = load_json("train","train_pub.json")
name_pubs = load_json("train","train_author.json")

# 采样组成验证集
np.random.seed = 1
choices = np.random.choice(len(name_pubs), size=10, replace=False)
keys = list(name_pubs.keys())
names = [keys[choice] for choice in choices]

In [26]:
import time 

def black_box(names, pubs_raw, 
              threshold=1.5, eps=0.2, min_samples=4, weight=1, weight2=0.1, 
              mode = 'train'):
    
    r_result = {}    # 返回的聚类作者集合
    result=[]        # 训练集上测试保存f1-score
    for n,name in tqdm_notebook(enumerate(names), total=len(names)):

        ilabel=0
        pubs=[] # all papers
        labels=[] # ground truth
        
        if(mode == 'train'):
            for author in name_pubs[name]:
                iauthor_pubs = name_pubs[name][author]
                for pub in iauthor_pubs:
                    pubs.append(pub)
                    labels.append(ilabel)
                ilabel += 1
        
        else:
            for cluster in name_pubs1[name]:
                pubs.extend(cluster)
        
        if len(pubs)==0:
            result.append(0)
            continue
        
        ##保存关系
        ###############################################################
        name_pubs_raw = {}
        for i,pid in enumerate(pubs):
            name_pubs_raw[pid] = pubs_raw[pid]
            
        dump_json(name_pubs_raw, 'genename', name+'.json', indent=4)
        save_relation(name+'.json', name)  
        ###############################################################
        
        
        
        ##元路径游走类
        ###############################################################r
        mpg = MetaPathGenerator()
        mpg.read_data("gene")
        ###############################################################
        
    
        
        ##论文关系表征向量
        ############################################################### 
        all_embs=[]
        rw_num = 5     # bagging，获得更优结果
        cp=set()
        for k in range(rw_num):
            mpg.generate_WMRW("gene/RW.txt",5,20) #生成路径集
            sentences = word2vec.Text8Corpus(r'gene/RW.txt')
            model = word2vec.Word2Vec(sentences, size=100,negative =25, min_count=1, window=10)
            embs=[]
            for i,pid in enumerate(pubs):
                if pid in model:
                    embs.append(model[pid])
                else:
                    cp.add(i)
                    embs.append(np.zeros(100))
            all_embs.append(embs)
        all_embs= np.array(all_embs)
        # print ('relational outlier:',cp)
        ############################################################### 

        
        
        ##论文文本表征向量
        ###############################################################   
        ptext_emb=load_data('gene','ptext_emb.pkl')
        tcp=load_data('gene','tcp.pkl')
        # print ('semantic outlier:',tcp)
        tembs=[]
        for i,pid in enumerate(pubs):
            tembs.append(ptext_emb[pid])
        ############################################################### 
        
        ##离散点
        outlier=set()
        for i in cp:
            outlier.add(i)
        for i in tcp:
            outlier.add(i)
        
        ##网络嵌入向量相似度
        sk_sim = np.zeros((len(pubs),len(pubs)))
        for k in range(rw_num):
            sk_sim = sk_sim + pairwise_distances(all_embs[k],metric="cosine")
        sk_sim =sk_sim/rw_num    
        
        ##文本相似度矩阵
        t_sim = pairwise_distances(tembs,metric="cosine")
        
        # 强规则匹配相似矩阵
        coau_sim = disambiguate_by_cluster(pubs, pubs_raw)

        w = weight     # 加权比例
        w2 = weight2
        sim = (sk_sim + w*t_sim+ w2*coau_sim)/(1+w+w2)
        
        
        ##evaluate
        ###############################################################
        pre = DBSCAN(eps = eps, min_samples = min_samples, 
                     metric ="precomputed").fit_predict(sim)
        
        for i in range(len(pre)):
            if pre[i]==-1:
                outlier.add(i)
        
        ## assign each outlier a label
        paper_pair = generate_pair(pubs,outlier)
        paper_pair1 = paper_pair.copy()
        K = len(set(pre))
        for i in range(len(pre)):
            if i not in outlier:
                continue
            j = np.argmax(paper_pair[i])

            while j in outlier:
                paper_pair[i][j]=-1
                j = np.argmax(paper_pair[i])
                if(i==j):
                    break
                
            if paper_pair[i][j]>=threshold:
                pre[i]=pre[j]
            else:
                pre[i]=K
                K=K+1
        
        ## find nodes in outlier is the same label or not
        for ii,i in enumerate(outlier):
            for jj,j in enumerate(outlier):
                if jj<=ii:
                    continue
                else:
                    if paper_pair1[i][j]>=threshold:
                        pre[j]=pre[i]

        if mode == 'test':
            r_result[name]=[]
            for i in set(pre):
                oneauthor=[]
                for idx,j in enumerate(pre):
                    if i == j:
                        oneauthor.append(pubs[idx])
                r_result[name].append(oneauthor)

        if mode == 'train':
            labels = np.array(labels)
            pre = np.array(pre)
            # print (labels,len(set(labels)))
            # print (pre,len(set(pre)))
            pairwise_precision, pairwise_recall, pairwise_f1 = pairwise_evaluate(labels,pre)
            # print (pairwise_precision, pairwise_recall, pairwise_f1)
            result.append(pairwise_f1)

    return np.mean(result) if mode=='train' else r_result


# test black box
f1_score = black_box(names, pubs_raw, threshold=1.5, eps=0.2, min_samples=4, weight=1)

print('BaseLine:', f1_score)

BaseLine: 0.6785717579137478


In [11]:
# 将模型看做一个黑箱，对选择的超参数进行Random Search优化
import pickle

params_list = []
params = {}   
best_f1 = 0.

num_iters = 100
for i in tqdm_notebook(range(num_iters), total=num_iters, desc='Random Search:'):
    threshold = np.random.uniform(0.5, 3)
    eps = np.random.uniform(0.01, 0.4)
    min_samples = int(np.random.uniform(2, 11))
    weight = np.random.uniform(0.1, 2.)
    weight2 = np.random.uniform(0.01, 0.3)
    
    # 启动模型
    f1_score = black_box(names, pubs_raw, 
                         threshold, eps, min_samples, weight, weight2)

    print('Iteration:', i, 'f1_score:', f1_score)

    # 多组参数
    if f1_score > 0.66:
        params['thresh'] = threshold
        params['eps'] = eps
        params['min_samples'] = min_samples
        params['weight'] = weight
        params['weight2'] = weight2
        params['best_f1'] = f1_score
        params_list.append(params)
        print(params)

    # update 
    if (f1_score > best_f1):
        best_f1 = f1_score
        params['thresh'] = threshold
        params['eps'] = eps
        params['min_samples'] = min_samples
        params['weight'] = weight
        params['weight2'] = weight2
        params['best_f1'] = f1_score
        print(params)

        # 序列化保存最佳参数
        with open("best_params.txt", "wb") as f:
            pickle.dump(params, f)


Iteration: 0 f1_score: 0.45972521199434635
{'thresh': 2.7946870392356797, 'eps': 0.321551358126279, 'min_samples': 2, 'weight': 0.7042424343737738, 'weight2': 0.11596993312379543, 'best_f1': 0.45972521199434635}


Iteration: 1 f1_score: 0.34563770677559763


Iteration: 2 f1_score: 0.3086973708897549


Iteration: 3 f1_score: 0.4144483108134608


Iteration: 4 f1_score: 0.27467053895692417


...

Iteration: 95 f1_score: 0.6941222711867336
{'thresh': 2.189330251740836, 'eps': 0.223234674555444, 'min_samples': 7, 'weight': 0.48636866093212483, 'weight2': 0.18453051671167192, 'best_f1': 0.6941222711867336}


Iteration: 96 f1_score: 0.277908274550836


Iteration: 97 f1_score: 0.34512888965408733


Iteration: 98 f1_score: 0.41468093754611235


Iteration: 99 f1_score: 0.6297276680565919


In [0]:
best_params = [{'thresh': 1.895702348705277, 'eps': 0.23323533280978634, 'min_samples': 10, 'weight': 1.6844036016493722, 'weight2': 0.12226499610495505, 'best_f1': 0.7228654623732524},
               {'thresh': 1.9212056928848318, 'eps': 0.17825504257031927, 'min_samples': 6, 'weight': 1.9976442732993076, 'weight2': 0.018512436017346732, 'best_f1': 0.7140424259203794},
               {'thresh': 1.9655230655959528, 'eps': 0.16199243027899998, 'min_samples': 8, 'weight': 0.7031722951284339, 'weight2': 0.10647427939776863, 'best_f1': 0.6814925631477007},
               {'thresh': 2.1091020048025606, 'eps': 0.28407270941126095, 'min_samples': 10, 'weight': 0.8811850075241879, 'weight2': 0.21315398382203724, 'best_f1': 0.6670736978167153},
               {'thresh': 2.660968437862895, 'eps': 0.17191900240196836, 'min_samples': 5, 'weight': 1.9711799579811513, 'weight2': 0.036972574101039275, 'best_f1': 0.7022727901794958},
               {'thresh': 1.9459681799441153, 'eps': 0.2762809057860908, 'min_samples': 9, 'weight': 0.462061241196594, 'weight2': 0.19950126048777347, 'best_f1': 0.67268473167475},
               {'thresh': 2.2734333434091187, 'eps': 0.2295660586677669, 'min_samples': 4, 'weight': 1.7421109932788073, 'weight2': 0.2531216850343555, 'best_f1': 0.7190989981434728},
               {'thresh': 1.4855482179230857, 'eps': 0.20240061054972353, 'min_samples': 9, 'weight': 1.1458205130112236, 'weight2': 0.16653627864758605, 'best_f1': 0.6765752734133532},
               {'thresh': 1.5298279361083482, 'eps': 0.16932030814718999, 'min_samples': 5, 'weight': 0.6956250719861805, 'weight2': 0.14827740681795953, 'best_f1': 0.6763060128587587},
               {'thresh': 1.8617883052232649, 'eps': 0.17627825684722714, 'min_samples': 5, 'weight': 1.0724393039581728, 'weight2': 0.09358893916943353, 'best_f1': 0.7039190235065002},
               {'thresh': 2.597386952567265, 'eps': 0.22784198830638125, 'min_samples': 5, 'weight': 0.8005773415839111, 'weight2': 0.22286756296266885, 'best_f1': 0.7046451614775233},
               {'thresh': 2.4811453769326697, 'eps': 0.2101849911360913, 'min_samples': 4, 'weight': 0.3872861708894282, 'weight2': 0.1779097557222204, 'best_f1': 0.6804992795113811},
               {'thresh': 1.7678767760189038, 'eps': 0.24871017017163022, 'min_samples': 8, 'weight': 1.591207887661069, 'weight2': 0.28282225246263876, 'best_f1': 0.7175080430524798},
               {'thresh': 2.0148861035957406, 'eps': 0.22416893801489832, 'min_samples': 4, 'weight': 1.3051080476372041, 'weight2': 0.28713480751218695, 'best_f1': 0.7142109396590666},
               {'thresh': 1.2432205674297494, 'eps': 0.23547656644439896, 'min_samples': 3, 'weight': 1.261599267590826, 'weight2': 0.2002268920288503, 'best_f1': 0.6849181101293793},
               {'thresh': 2.189330251740836, 'eps': 0.223234674555444, 'min_samples': 7, 'weight': 0.48636866093212483, 'weight2': 0.18453051671167192, 'best_f1': 0.6941222711867336}]

# 序列化保存
with open("best_params.txt", "wb") as f:
            pickle.dump(best_params, f)

In [19]:
# 加载最优params

import pickle

with open("best_params.txt", "rb") as f:
    best_params = pickle.load(f)

# print(best_params)

[{'thresh': 1.895702348705277, 'eps': 0.23323533280978634, 'min_samples': 10, 'weight': 1.6844036016493722, 'weight2': 0.12226499610495505, 'best_f1': 0.7228654623732524}, {'thresh': 1.9212056928848318, 'eps': 0.17825504257031927, 'min_samples': 6, 'weight': 1.9976442732993076, 'weight2': 0.018512436017346732, 'best_f1': 0.7140424259203794}, {'thresh': 1.9655230655959528, 'eps': 0.16199243027899998, 'min_samples': 8, 'weight': 0.7031722951284339, 'weight2': 0.10647427939776863, 'best_f1': 0.6814925631477007}, {'thresh': 2.1091020048025606, 'eps': 0.28407270941126095, 'min_samples': 10, 'weight': 0.8811850075241879, 'weight2': 0.21315398382203724, 'best_f1': 0.6670736978167153}, {'thresh': 2.660968437862895, 'eps': 0.17191900240196836, 'min_samples': 5, 'weight': 1.9711799579811513, 'weight2': 0.036972574101039275, 'best_f1': 0.7022727901794958}, {'thresh': 1.9459681799441153, 'eps': 0.2762809057860908, 'min_samples': 9, 'weight': 0.462061241196594, 'weight2': 0.19950126048777347, 'best

In [23]:
sorted_best_params = sorted(best_params, key=lambda params: -params['best_f1'])
print(sorted_best_params[:3])

[{'thresh': 1.895702348705277, 'eps': 0.23323533280978634, 'min_samples': 10, 'weight': 1.6844036016493722, 'weight2': 0.12226499610495505, 'best_f1': 0.7228654623732524}, {'thresh': 2.2734333434091187, 'eps': 0.2295660586677669, 'min_samples': 4, 'weight': 1.7421109932788073, 'weight2': 0.2531216850343555, 'best_f1': 0.7190989981434728}, {'thresh': 1.7678767760189038, 'eps': 0.24871017017163022, 'min_samples': 8, 'weight': 1.591207887661069, 'weight2': 0.28282225246263876, 'best_f1': 0.7175080430524798}]


#### 4. 在测试集上生成最佳结果

In [0]:
import re
from gensim.models import word2vec
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

# 加载数据
pubs_raw = load_json("sna_data","sna_valid_pub.json")
name_pubs1 = load_json("sna_data","sna_valid_example_evaluation_scratch.json") 
names = list(name_pubs1.keys())

In [34]:
for i, params in enumerate(sorted_best_params):
    if i<=5:
        continue
    if i>=10:
        break
    threshold = params['thresh']
    eps = params['eps']
    min_samples = int(params['min_samples'])
    weight = params['weight']
    weight2 = params['weight2']
    print('Estimate f1-score:', params['best_f1'])
    r_result = black_box(names, pubs_raw, 
                         threshold, eps, min_samples, weight, weight2, mode='test')
    dump_json(r_result, "results", "enhancedword2vec_dbscan_"+str(i)+".json",indent =4)


Estimate f1-score: 0.7039190235065002


Estimate f1-score: 0.7022727901794958


KeyboardInterrupt: ignored

In [36]:
# default
r_result = black_box(names, pubs_raw, mode='test', weight2=0., threshold=1.6, min_samples=6)
dump_json(r_result, "results", "enhancedword2vec_dbscan_"+'default'+".json",indent =4)